# Actual evapotranspiration - using the right surface parameters

## Intro
In principle, the Penman-Monteith method is a sound physical model of transpiration. If we would like to use the Penman-Monteith method to estimate actual evapotranspiration rather than reference evapotranspiration (as done in Practical 3b) we need to make sure that the parameters used in the Penman-Monteith equation are representative of the current conditions. Here we focus on 
* albedo
* roughness length
* canopy resistance 

Since the process of evapotranspiration is an instantaneous process with a strong diurnal cycle we need to use data with an averaging interval that enables to resolved that diurnal cycle. Therefore this practical will be based on 30-minute average fluxes.

As usual, this practical comes with an [answer sheet]().

## The data
The data that you will use come from the same dataset as used in Practical-1, but with the difference that we now use 30-minute averages.

## Initialize Python stuff and read the data
Please run the cell below by selecting it and pressing Shift+Enter. Or Press the Run button in the toolbar at the top of the screen (with the right pointing triangle).

In [1]:
# Load some necessary Python modules
import pandas as pd # Pandas is a library for data analysis
pd.set_option("mode.chained_assignment", None)
import numpy as np # Numpy is a library for processing multi-dimensional datasets
from hupsel_helper import myplot, myreadfile
from hupsel_helper import f_Lv, f_esat, f_s, f_gamma, f_cp, f_cos_zenith, f_atm_transmissivity, f_ra

With the commands above, the following functions have become available:
* `f_Lv(T)`: compute latent heat of vapourization from temperature (in K)
* `f_esat(T)`: compute saturated vapour pressure from temperature (in K)
* `f_s(T)`: compute the slope of the saturated vapour pressure as a function of temperature (in K)
* `f_gamma(T, p, q)`: compute the psychrometer constant from temperature (K), pressure (Pa) and specific humidity (kg/kg)
* `f_cp(q)`: compute the specific heat of air (in J/kg/K) using specific humidity (in kg/kg)

We have added some new possibilities to the `myplot` function (already introduced in practical 3a):
* you can now manually set the label text on the x-axis and y-axis (in particular relevant if you do not plot from a dataframe): `myplot( [x, y], xlabel='wind speed (m/s)', ylabel='temperature (K)')`
* you can now manually set the name of a series, to be used in the legend (again, relevant if no dataframe is used): `myplot( [x, y, 'o', 'my specially constructed variable'] )`. Note that in this case you *should* specify the type of plotting symbol (here a dot: `'o'`, could also be `'-'` for line and `'#'` for a bar graph).

Now read the 30-minute data for 2014 from the Excel file.

In [2]:
# File name: this is a different file that you worked on before
fname='Hupsel2014_MeteoData.xlsx'

# Get the data
df = myreadfile(fname, type='30min')

## Albedo
When modelling net radiation (as is done in the FAO method) an essential parameter is the albedo. The FAO-method assumes a value of 0.23. But if you want to use the Penman-Monteith equation to determine actual ET for an actual surface, you need to know the real albedo. 

From the measurements we did in the field we know that the albedo can be variable between land-use types and within fields. But it can also vary with time as a result of changes in surface conditions and changes in radiation properties (solar zenith angle and cloudiness).

### <span style='background:lightblue'>Question 1</span>
Compute the albedo and investigate how it varies with time:
* throughout the experiment (plot as a function of `df['Date']`)
* throughout the day (taking all days together: plot as a function of `df['Time']` (plot with *dots* (i.e. `'o'`), not lines))

You may need to zoom in a bit to ignore extreme values.

In [3]:

# Hints for us, not for students
#albedo = df['Kout']/df['Kin.1']
#myplot( [df['Date'], albedo], xlabel='Time', ylabel='albedo (-)')


We see that the albedo varies during the day. This variation could be related to a dependence of albedo on solar zenith angle (so it is not so much the time that is relevant). The cosine of the solar zenith angle ( $\cos (\theta_z)$ ) can be obtained with the function `f_cos_zenith`:
```
cos_zenith_angle = f_cos_zenith(date_time, latitude, longitude)

```
where `date_time` is an array with time stamps (simply use `df['Date']` for that) and `latitde` and `longitude` are the coordinates in degrees. The Hupsel KNMI station is located at latitude = 52.0675 and longitude = 6.6567).

### <span style='background:lightblue'>Question 2</span>
Determine the $\cos (\theta_z)$ for your data. What is the range of values you expect for $\cos (\theta_z)$? 
Plot the albedo (y-axis) as a function of $\cos (\theta_z)$. Explain/interpret the relationship that you see.

In [4]:
# Location of Hupsel weather station
latitude = 52.0675
longitude = 6.6567

# Hints for us, not for students
cos_zenith_angle = f_cos_zenith(df['Date'], latitude, longitude)
albedo = df['K_out_m']/df['K_in_m']
myplot( [cos_zenith_angle, albedo, 'o'], xlabel='cos(zenith angle)', ylabel='albedo (-)')

Loading BokehJS ...

The dependence of the reflectivity of a surface on the direction of the solar radiation only has an effect on the overall albedo if the radiation comes mainly from one direction. So when the radiation is diffuse (under cloudy conditions), one would expect the albedo to be mostly *independent* of $\cos ( \theta_z )$ (see e.g. figure 2.10 in Moene & Van Dam (2014)). 

To test this, we would need information the amount of diffuse radiation. This info is not available directly, but there are two variables that could be helpful here:
* the sunshine duration within the 30 minute interval (`df['sun_dur']`)
* the transmissivity of the atmosphere $\tau_b = \frac{K^\downarrow}{K_0}$ where $K_0$ is the radiation at the top of the atmosphere.

The transmissivity of the atmosphere can be computed with the function `f_atm_transmissivity`:
```
trans = f_atm_transmissivity(date_time, latitude, longitude, K_in)
```
where `date_time` is an array with time stamps (`df['Date']`), `latitde` and `longitude` are the coordinates in degrees and `K_in` is global radiation. 

### <span style='background:lightblue'>Question 3</span>
* What values for sunshine duration or atmospheric transmissivity do you expect for cloudy versus sunny conditions?
* Make the same plot as under question 2 (albedo vs. $\cos(\theta_z)$), but now color the dots by one of the variable that would help to distingish between mostly diffuse conditions and conditions with mainly direct radiation. Use the additional keyword `color_by` in the plot command: `myplot([.. , .., 'o'], color_by = df['sun_dur'])` (in stead of `df['sun_dur']` you could also use the variable in which you stored the transmissivity, e.g. `trans`).
* Does the dependence of albedo on $\cos(\theta_z)$ differ between cloudy and sunny conditions? If so, how?

In [5]:
# Hints for us, not for students
trans = f_atm_transmissivity(df['Date'], latitude, longitude, df['K_in_m'])
myplot([cos_zenith_angle,albedo,'o'], color_by=df['sun_dur'])
myplot([cos_zenith_angle,albedo,'o'], color_by=trans)

Loading BokehJS ...

Loading BokehJS ...

## Roughness length

To derive the roughness lengths for momentum ($z_0$) and heat ($z_{0h}$)from observations we need to consider the effect of stability on the wind profiles (equation (3.42) in the AVSI book):
$$
 \overline{u}(z_u) = \frac{u_*}{\kappa} \left[ \ln\left(\frac{z_u}{z_0}\right) - 
                                               \Psi_m\left(\frac{z_u}{L}\right) + 
                                               \Psi_m\left(\frac{z_0}{L}\right) \right]
$$

However, to obtain $z_0$ from the expressions for the profiles would require quite some programming. An easier way is to start with a two-step method for $z_0$:

* First compute the roughness length for momentum from observed $\overline{u}$ and $u_*$  for each data point, assuming neutral conditions (i.e. $\frac{z}{L} \approx 0$). Then, the computation of the roughness length is a matter of rewriting the expression for the logarithmic wind profile (remember that the Python functions that you might need are `np.log(x)` and `np.exp(x)`).
* Filter the data such that only the most neutral data are retained:
  * Simple, less exact: Use high wind speeds as an indication neutral conditions: plot $z_0$ versus wind speed to see where the neutral data are. (i.e. $z_0$ on the y-axis, wind speed on the horizontal axis).
  * Complex, more exact: Use the $\frac{z}{L}$ stability parameters as an indication for neutral conditions: plot $z_0$ versus $\frac{z}{L}$ and zoom in on the neutral part.

What we call 'filtering' above can simply be done by plotting: plot all $z_0$ values and then search for that part of the plot where you expect neutral conditions. From that part of the graph you can *estimate* the roughness length (within an order of magnitude, but that is enough). Usually it helps to use a log-scale for the $z_0$–axis because the spread in values is quite large.

As the KNMI station was surrounded by grass you can assume that the roughness length you derive is representative for a grass meadow. The question to be answered is: what is the value of $z_0$ for the grass and is the value you find for $z_0$ very different from the values used in the FAO method?  


### <span style='background:lightblue'>Question 4</span>
Determine the roughness length for momentum from the current dataset:
* compute the value assuming neutral condition
* plot it against a variable that you can use to select neutral conditions (see above: wind speed or stability)
* the value may be affected by the upstream conditions: so color your plot with wind direction

The plot you get will at first seem quite chaotic. But by zooming and panning you will get a reasonable view on the values (use a range between $10^{-3}$ and $10^{1}$ m for $z_0$ and 0-10 m/s for wind speed and -10 to 10 for $\frac{z}{L}$ (you can easily adjust those by panning (sitch on with button right of graph with arrows) and scroll-zoom (switch on with button with mouse and magnifying glass: then position cursur over axis you want to zoom and roll your mouse wheel).
Is this a reasonable value? How does it compare to the value that is assumed in the FAO method for reference ET? Are some of the values affected by upstream conditions (see map below).
<img src="surrounding_KNMI_station.png" width="80%">

In [6]:
# Use this cell to compute the roughness length for all data and determine the correct value based on 
# some form of selection
# 
# Hints for us, not for students
# Method 1: plot against wind speed, look at the z0 values at hight wind speeds.
zu = 10
karman = 0.4
z0 = zu / np.exp((karman*df['u_10_m'] / df['ustar_m'] ))


myplot( [df['u_10'], z0, 'o'], xlabel='wind speed (m/s)', ylabel='z0 (m)', \
         y_axis_type='log', color_by=df['u_dir'])

# Method 2: plot against z/L, look at the z0 values at neutral conditions (z/L = 0)
u_star = df['ustar_m']
T = df['T_1_5'] + 273.15
wT = df['H_m']/(df['rho']*f_cp(df['q']))
g = 9.81 
z_L = karman*g*zu*wT/((u_star**3)*T)
y = (karman*df['u_10_m'] / df['ustar_m'] )
myplot( [z_L, z0, 'o'], xlabel='z/L (-)', ylabel='z0 (m)', \
         y_axis_type='log', color_by=df['DOY'])


Loading BokehJS ...

Loading BokehJS ...

## Canopy resistance
If both the actual evapotranspiration is measured as well as all input variables for the Penman-Monteith equation (Q*, G, T, e, ra), then the canopy resistance can be obtained. Inversion of the Penman-Monteith yields for rc the following explicit expression:  
$$
r_c=r_a \left[ \frac{s(Q^*-G)+\frac{\rho c_p}{r_a} \left(e_s (T_a)-e_a \right) )}{\gamma L_v E}-\frac{s}{\gamma}-1 \right]
$$

### <span style='background:lightblue'>Question 5</span>
Compute the canopy resistance for each data point. In your analysis focus on:
* The diurnal cycle (how does $r_c$ vary through the day)
* The development over time of the midday value of the canopy resistance (are there periods of significantly higher or lower values, perhaps linked to periods of soil moisture stress, wet canopy etc.)
* Compare the values you find to those prescribed by the FAO.

Note: The data may be quite noisy, so it might help to use a logarithmic axis for $r_c$. If the plot does not auto-scale well, zoom in or out so that the y-axis runs between $10^{-1}$ and $10^{4}$.


In [7]:
# Hints for us, not for students
zu = 10 # m
zT = 1.5 # m
d  = 0 # m
z0 = 0.02 # m (best guess from our own data?)
z0h = 0.01 * z0
ra = f_ra(df['u_10'], zu, zT, d, z0, z0h)
T = df['T_1_5'] + 273.15
p = df['p']*100
q = df['q']
s = f_s(T)
esat = f_esat(T)

gamma = f_gamma(T, p, q)
cp    = f_cp(q)
Qnet = df['Q_net_m']
G    = df['G_0_m']
LvE  = df['LvE_m']
ea   = df['e']
rho  = df['rho']

numer1 = s*(Qnet - G)
numer2 = (rho*cp/ra)*(esat - ea)
rc = ra *( (numer1 + numer2)/(gamma*LvE) - (s/gamma) - 1)

myplot([df['Time'], rc, 'o'], y_axis_type='log', \
       xlabel = 'time of day (hour)', ylabel = 'canopy resistance (s/m)', color_by=df['DOY'])


Loading BokehJS ...

## Photosynthesis
The data set also contains information about the exchange of CO2 between the plants and the atmosphere. The eddy-covariance system directly measures the net ecosystem exchange (NEE): the net effect of uptake by photosynthesis (GPP, gross primary production) and release due to respiration. The portioning of NEE over respiration (TER) and GPP cannot be measured but has been estimated.

### <span style='background:lightblue'>Question 6</span>
Explore the NEE flux (variable `FCO2_m`). How does it very as a function of time of day (variable `Time`)?

In [21]:
# Hints for us, not for students
# Typical diurnal cycle of GPP
myplot([df['Time'],df['FCO2_m'], 'o'], \
       xlabel = 'time of day (hour)', ylabel = 'GPP (kg/m^2/s)', color_by=df['DOY'])

Loading BokehJS ...

Since we are interested in the functioning of the plants we focus on the photosynthesis-related flux GPP.  The photosynthesis is related to light interception. 

An important concept in is the light response curve (LRC, how much CO2 uptake takes place at which light level). 

### <span style='background:lightblue'>Question 6</span>
Construct a light response curve by making a scatter plot of GPP (y-axis) versus global radiation (as a proxy for the photosynthetically active radiation (you may need to tweak the axes to reduce the effect of outliers).
* Focus on the general shape (initial slope at low light levels and maximum assimilation at high light levels).
* There seems to be quite some spread. Is this noise, or is there a signal. Check the light response curves of individual days (by filtering the data by DOY). Estimate level of the plateau in the light response curve (the maximum assimilation) for about 10 days (note them down) and try to find a relationship between that maximum value and conditions during those days.

In [22]:
# Light response curve
# colored by DOY (mowing in last days)
myplot([df['K_in_m'],df['GPP'], 'o'], \
       xlabel = 'global radiation (W/m2)', ylabel = 'GPP (kg/m^2/s)', color_by=df['DOY'])

# Colored by transmissivity as a proxy for direct radiation (vs diffuse)
trans = f_atm_transmissivity(df['Date'], latitude, longitude, df['K_in_m'])
myplot([df['K_in_m'],df['GPP'], 'o'], \
       xlabel = 'global radiation (W/m2)', ylabel = 'GPP (kg/m^2/s)', color_by=trans)

# Water use efficiency
myplot([df['LvE_m'],df['GPP'], 'o'], \
        xlabel = 'latent heat flux (W/m2)', ylabel = 'GPP (kg/m^2/s)',  color_by=df['DOY'])


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Another way of quantifying the relation between supplied light energy and resulting carbon uptake is the light use efficiency (LUE): the ratio of GPP over radiation input for a given amount of radiation (so you do not look at the LRC as a curve, but at an individual point) (again use global radiation as a proxy for PAR).

### <span style='background:lightblue'>Question 7</span>
Compute for each data point the light use efficiency. There are various ways to look at it:
* Plot the LUE as a function of time over the entire experiment. How does the LUE vary from day to day? To simplify the analysis, you could filter the data to only show midday values.
* Plot the LUE as a function of time on the day to see the average diurnal cycle. What does it look like could you explain it.
* Does the LUE vary with meteorological variables (e.g. relative humidity, VPD, temperature ….)?

In [28]:
# Light use efficiency
LUE = df['GPP']/df['K_in']

myplot([df['Date'],LUE, 'o'], \
        xlabel = 'time', ylabel = 'light use efficiency (kg/J)',  color_by=df['RH_1_5'])

myplot([df['Time'],LUE, 'o'], \
        xlabel = 'time of day', ylabel = 'light use efficiency (kg/J)',  color_by=df['DOY'])

Loading BokehJS ...

Loading BokehJS ...

Finally, the uptake of CO2 and the transpiration are closely coupled via the stomata. In that respect an important variable is the water use efficiency (WUE): amount of CO2 uptake for a given amount of water loss (less water use per carbon uptake means higher efficiency).

### <span style='background:lightblue'>Question 7</span>
Analyse the WUE in a similar way as the light use efficiency (variation between days, variation through the day, and dependence on other variables). It is good enough to use LvE  as proxy for the amount of water lost (i.e.: WUE = LvE / GPP).  You could also analyse in the form of a ‘LvE response curve’: GPP as a function of L¬vE.  Such a curve could answer the question: does additional evapotranspiration lead to additional CO2 uptake or does it level off?


In [23]:
# Water use efficiency
myplot([df['LvE_m'],df['GPP'], 'o'], \
        xlabel = 'latent heat flux (W/m2)', ylabel = 'GPP (kg/m^2/s)',  color_by=df['DOY'])


Loading BokehJS ...


### <span style='background:lightblue'>Question last</span>
Now upload your answer document(Word file) to Brightspace.

## Conclusion
